In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import time
import warnings
warnings.filterwarnings("ignore")

In [19]:
query = '''

-- 여기부터 회차 테이블 작성 쿼리--

WITH t1 AS 
(SELECT DISTINCT a.s25csq -- 계약번호 
     , b.kunnr
	 , a.custname -- 상호 (hashing처리되니까 현재로서는 어차피 무의미함)
     , a.listid
	 , a.contactstartdate
	 , a.contactstarttime
	 , a.dialresultcd
  FROM telepro.tb_contact a
  LEFT JOIN (SELECT DISTINCT ctnum, kunnr FROM sap.zsdt4013) b
         ON a.s25csq = b.ctnum
 WHERE a.listid IN (SELECT DISTINCT listid FROM default.tm_list_header))
 
, t2 AS
(SELECT *
     , CASE WHEN contactstartdate = '' THEN '9999/99/99' ELSE contactstartdate END AS contactstartdate_new
     , DENSE_RANK() OVER (PARTITION BY s25csq ORDER BY s25csq, listid) AS nth_contact_ctnum
	 , ROW_NUMBER() OVER (PARTITION BY s25csq, listid ORDER BY contactstartdate DESC, contactstarttime DESC) AS row_num_ctnum
     , DENSE_RANK() OVER (PARTITION BY kunnr ORDER BY kunnr, listid) AS nth_contact_kunnr
	 , ROW_NUMBER() OVER (PARTITION BY kunnr, listid ORDER BY contactstartdate DESC, contactstarttime DESC) AS row_num_kunnr
   FROM t1)
   
, t3 AS
(SELECT *
     , LAG(nth_contact_final_date_ctnum) OVER (PARTITION BY s25csq ORDER BY s25csq, listid) AS prior_contact_final_date_ctnum
     , LAG(nth_contact_final_date_kunnr) OVER (PARTITION BY kunnr ORDER BY kunnr, listid) AS prior_contact_final_date_kunnr    
  FROM (SELECT *
             , MAX(contactstartdate) OVER (PARTITION BY s25csq, listid ORDER BY s25csq, listid) AS nth_contact_final_date_ctnum
	         , MIN(contactstartdate_new) OVER (PARTITION BY s25csq, listid ORDER BY s25csq, listid) AS nth_contact_first_date_ctnum
             , MAX(contactstartdate) OVER (PARTITION BY kunnr, listid ORDER BY kunnr, listid) AS nth_contact_final_date_kunnr
	         , MIN(contactstartdate_new) OVER (PARTITION BY kunnr, listid ORDER BY kunnr, listid) AS nth_contact_first_date_kunnr	         
          FROM t2)
ORDER BY s25csq, kunnr, listid, contactstartdate, contactstarttime)

, t4 AS (
SELECT a.s25csq
     , b.ctnum AS custid
     , MAX(nth_contact_ctnum) AS nth_contact
  FROM t3 a
LEFT JOIN (SELECT ctnum, ctnum_hash FROM adt_work.hash_info) b
      ON a.s25csq = b.ctnum_hash
GROUP BY a.s25csq, b.ctnum
) 
 
--여기까지 회차 테이블 작성 쿼리-- 

, t5 AS (
SELECT DISTINCT a.listseqid 
     , a.listid
     , b.listname
     , CASE WHEN listname LIKE '%정전%' THEN '정전'
           WHEN listname LIKE '%주택%' THEN '주택'
		   ELSE '화재' END AS listtype
     , b.extracteddate
	 , CASE WHEN b.extracteddate LIKE '2020/09%' THEN 1 ELSE 0 END AS this_month_list -- 월 변경
	 , d.nth_contact
     , a.custid
     , a.resultcd
     , a.dialdate
     , a.dialtime
     , c.contactstartdate
     , c.contactstarttime
     , c.contactenddate
     , c.contactendtime
     , CASE WHEN a.resultcd IN ('NF03', 'SE05', '') THEN 0 ELSE 1 END AS call_success
     , CASE WHEN a.resultcd = 'SS01' THEN 1 ELSE 0 END AS final_success
  FROM (SELECT DISTINCT listseqid, listid, custid, dialdate, dialtime, resultcd FROM default.tm_list_detail) a
 LEFT JOIN (SELECT DISTINCT listid, listname, extracteddate FROM default.tm_list_header) b 
        ON a.listid = b.listid
 LEFT JOIN (SELECT DISTINCT listseqid, listid, contactstartdate, contactstarttime, contactenddate, contactendtime
              FROM telepro.tb_contact) c
        ON a.listseqid = c.listseqid
 LEFT JOIN (SELECT custid, nth_contact FROM t4) d
        ON a.custid = d.custid
 WHERE a.dialdate LIKE '2020/09%' -- 월 변경
)

 -- 마감 실적 뽑기
 
 SELECT listtype
     , this_month_list
	 , nth_contact
     , COUNT(DISTINCT custid) AS cust_cnt
     , COUNT(custid) AS call_cnt
     , COUNT(DISTINCT CASE WHEN call_success = 1 THEN custid ELSE NULL END) AS call_success_cnt
     , COUNT(DISTINCT CASE WHEN final_success = 1 THEN custid ELSE NULL END) AS final_success_cnt
  FROM t5
GROUP BY listtype, this_month_list, nth_contact
ORDER BY listtype, this_month_list, nth_contact 

'''

In [20]:
from skinet.utils.athena_tools import *
conn = athena_connection()
df = load_from_athena(conn, query)

...tmp_db name = tmp_20201006_09_41_32_68b67fc9319906353a1d48c519366b1841ad9bf4
Start creating table
drop table if exists default.tmp_20201006_09_41_32_68b67fc9319906353a1d48c519366b1841ad9bf4
<< table not exist

        create  table         default.tmp_20201006_09_41_32_68b67fc9319906353a1d48c519366b1841ad9bf4
        with (format='Parquet', parquet_compression='GZIP')
        as
        

-- 여기부터 회차 테이블 작성 쿼리--

WITH t1 AS 
(SELECT DISTINCT a.s25csq -- 계약번호 
     , b.kunnr
	 , a.custname -- 상호 (hashing처리되니까 현재로서는 어차피 무의미함)
     , a.listid
	 , a.contactstartdate
	 , a.contactstarttime
	 , a.dialresultcd
  FROM telepro.tb_contact a
  LEFT JOIN (SELECT DISTINCT ctnum, kunnr FROM sap.zsdt4013) b
         ON a.s25csq = b.ctnum
 WHERE a.listid IN (SELECT DISTINCT listid FROM default.tm_list_header))
 
, t2 AS
(SELECT *
     , CASE WHEN contactstartdate = '' THEN '9999/99/99' ELSE contactstartdate END AS contactstartdate_new
     , DENSE_RANK() OVER (PARTITION BY s25csq ORDER BY s25csq, li

In [21]:
org = df.copy()

In [22]:
df = org.copy()

In [23]:
df['nth_contact'].fillna(1, inplace=True)
df['nth_contact'] = df['nth_contact'].astype(int)
df['nth_contact'] = df['nth_contact'].astype(str)

In [24]:
df['nth_contact'] = np.where(df['nth_contact'] > '5', '5회차 이상', df['nth_contact'])

In [25]:
df['this_month_list'] = df['this_month_list'].map({0:"기 배분", 1:"신규 배분"})

In [26]:
df.groupby(by=['listtype', 'this_month_list', 'nth_contact']).sum().to_csv('sample.csv', encoding='utf-8-sig')

In [27]:
df.groupby(by=['listtype', 'this_month_list', 'nth_contact']).sum()

cust_cnt  call_cnt  call_success_cnt  \
listtype this_month_list nth_contact                                         
정전       기 배분            1                 149       775                88   
                         2                  66       277                40   
                         3                  37       134                20   
                         4                  17        49                10   
                         5                  33       105                16   
                         5회차 이상             16        43                 8   
         신규 배분           1                  84       185                63   
                         2                  33        71                25   
                         3                  56        70                37   
                         4                  52        72                25   
                         5                  57        74                37   
                         5회차 이상             62        88                37   
주택       기 배분            1                 385      2008               209   
                         2                 274      1008               141   
                         3                   1         4                 0   
         신규 배분           1                 401       914               317   
                         2                1839      3265              1185   
                         3                  13        30                11   
                         4                   2        11                 1   
                         5                   1         1                 1   
화재       기 배분            1                6794     19413              4335   
                         2                1485      5480               866   
                         3                1782      4178              1018   
                         4                1440      3477               811   
                         5                1383      3244               786   
                         5회차 이상            983      2091               583   
         신규 배분           1                2651      5633              2004   
                         2                1502      2810              1093   
                         3                1667      2552              1100   
                         4                1492      2331               935   
                         5                1738      2790              1089   
                         5회차 이상           1412      2255               906   

                                      final_success_cnt  
listtype this_month_list nth_contact                     
정전       기 배분            1                            8  
                         2                            3  
                         3                            0  
                         4                            0  
                         5                            2  
                         5회차 이상                       0  
         신규 배분           1                           16  
                         2                            4  
                         3                            4  
                         4                            2  
                         5                            2  
                         5회차 이상                       1  
주택       기 배분            1                           12  
                         2                            8  
                         3                            0  
         신규 배분           1                           66  
                         2                          103  
                         3                            2  
                         4                            0  
                         5                            0  
화재       기 배분            1                          603  
                         2                         